## 러너블 표준 인터페이스
- invoke(): 단일 입력을 처리하여 결과를 반환하는 동기 메서드
- batch(): 여러 입력을 동시에 처리하는 동기 메서드
- stream(): 결과를 스트리밍 방식으로 반환하는 동기 메서드
- ainvoke(): invoke()의 비동기 버전
- abatch(): batch()의 비동기 버전
- astream(): stream()의 비동기 버전
- astream_log(): 중간 단계와 최종 결과를 비동기적으로 스트리밍
- astream_events(): 체인에서 발생하는 이벤트를 비동기적으로 스트리밍

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# 오픈AI의 대규모 언어 모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
# 프롬프트 템플릿 정의: 주어진 주제에 대한 설명 요청
prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧은 설명을 해주세요.")
# 출력 파서 정의: AI 메시지의 출력 내용을 추출
parser = StrOutputParser()
# 프롬프트, 모델, 출력 파서를 체인으로 연결
chain = prompt | model | parser

In [4]:
# 응답 호출
chain.invoke({"topic": "더블딥"})

'더블딥(Double Dip)은 경제학 용어로, 경기가 한 번 회복된 후 다시 침체되는 현상을 말합니다. 일반적으로 경기 침체가 끝나고 경제가 회복되는 과정에서 잠시 긍정적인 신호가 나타나지만, 이후 다시 악화되어 두 번째 침체가 발생하는 경우를 의미합니다. 이러한 현상은 경제 정책의 명확한 효과가 부족하거나 외부적인 충격이 있을 때 자주 발생할 수 있습니다. 더블딥은 기업 투자와 소비자 신뢰에 부정적인 영향을 미쳐 경제 회복을 더욱 어렵게 만들 수 있습니다.'

batch(): 배치작업은 다수의 입력을 한꺼번에 처리하는 방식으로 성능을 최적화 하거나 여러개의 작업을 동시에 처리해야할 때 유용함.

In [5]:
# 주어진 주제 리스트에 대한 응답을 배치로 출력
chain.batch([{"topic": "더블딥"},{"topic": "인플레이션"}])

['더블딥(Double Dip)은 경제학에서 사용되는 용어로, 경기 침체가 발생한 후에 잠시 회복되었다가 다시 한 번 침체에 빠지는 상황을 가리킵니다. 일반적으로 경제가 회복세를 보인다고 해도, 여러 요인으로 인해 다시 마이너스 성장으로 돌아가는 경우에 해당합니다. 이러한 패턴은 소비자 신뢰도 하락, 고용 시장의 불안정 등으로 인해 발생할 수 있으며, 경제 정책 결정자들에게는 큰 도전 과제가 됩니다. 더블딥은 종종 경제 사이클의 불확실성을 나타내는 지표로 여겨집니다.',
 '인플레이션은 일반적으로 물가가 상승하는 현상을 의미합니다. 이는 경제 내에서 상품과 서비스의 평균 가격 수준이 지속적으로 상승하는 것으로, 화폐의 구매력이 감소하게 됩니다. 인플레이션은 다양한 요인에 의해 발생할 수 있으며, 수요 증가, 생산 비용 상승, 통화 공급의 증가 등이 이에 포함됩니다. 적당한 수준의 인플레이션은 경제 성장에 긍정적인 영향을 미칠 수 있지만, 과도한 인플레이션은 경제 불안정을 초래하고 소비자와 기업에 부정적인 영향을 미칠 수 있습니다. 이를 제어하기 위해 중앙은행은 금리 조정 등의 통화 정책을 활용합니다.']

- stream(): 모델의 응답을 토큰 단위로 스트리밍하여 반환
- flush=True: 출력 버퍼를 즉시 플러시하여 결과를 지연 없이 실시간으로 화면에 보여줌
  - 대기시간이 중요한 작업이나 실시간 피드백이 필요한 응용 프로그램에서 유용

In [6]:
# 응답을 토큰 단위로 스트리밍하여 출력
for token in chain.stream({"topic": "더블딥"}):
    #스트리밍된 내용을 출력, 각 내용을 출력하며 버퍼를 즉시 플러시하여 실시간으로 보여줌
    print(token, end="", flush=True)

더블딥(Double Dip)은 경제학 및 금융 시장에서 사용되는 용어로, 경기 침체가 한 차례 일어난 후 회복되다가 다시 한 번 추가적인 경기 침체를 경험하는 현상을 의미합니다. 즉, 경제가 일시적으로 회복되는 것처럼 보이다가 다시 하강하는 형태입니다. 이러한 상황은 일반적으로 소비자 신뢰, 투자 및 고용 시장의 불확실성으로 인해 발생할 수 있습니다. 더블딥은 경제 정책 입안자들에게 큰 도전과제를 안겨주며, 장기적인 경제 회복에 부정적인 영향을 미칠 수 있습니다.

### 러너블을 체인으로 연결하는 방법

In [7]:
# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# 이 대답을 영어로 번역해주세요 라는 질문을 생성하는 프롬프트 템플릿 정의
analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해주세요: {answer}")

In [8]:
# 이전에 정의된 체인(chain)과 새로운 작업을 연결하는 체인 구성
composed_chain = {"answer": chain} | analysis_prompt | model | StrOutputParser()
# "더블딥" 이라는 주제로 응답을 생성하고 체인 실행
composed_chain.invoke({"topic": "더블딥"})

'The term "double dip" is used in economics to describe a phenomenon where the economy experiences a temporary recovery after a recession, only to fall back into another recession. Typically, signs of recovery appear after the initial downturn, but various factors can cause the economy to decline again. Therefore, a double dip indicates that two recessions happen consecutively. Disruptions in economic policy or external shocks can exacerbate this phenomenon.'

이전에 생성된 체인이 실행되고 "answer"라는 키로 영어로 번역하는 프롬프트로 전달되고, 오픈AI 모델을 실행하여 답변을 생성하고 결과를 StrOutputParser로 문자열로 변환

In [9]:
# 이전에 정의된 값들
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧은 설명을 해주세요.")
chain = prompt | model | StrOutputParser()
analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해주세요: {answer}")

#람다 함수를 사용한 체인 구성
composed_chain_with_lambda = (
    #이전에 정의된 chain을 사용하여 입력된 데이터를 받아옵니다.
    chain
    #입력된 데이터를 "answer"키로 변환하는 람다 함수를 적용합니다.
    | (lambda input: {"answer": input})
    # "answer" 키를 가진 데이터를 영어로 번역하도록 프롬프트에 전달합니다.
    | analysis_prompt
    # 프롬프트에서 생성된 요청을 모델에 전달하여 결과를 생성합니다.
    | model
    # 모델에서 반환된 결과를 문자열로 파싱합니다.
    | StrOutputParser()
)
# "더블딥"이라는 주제로 답변을 생성하고, 답변을 영어로 번역합니다.
composed_chain_with_lambda.invoke({"topic": "더블딥"})

'The term "Double Dip" is primarily used in the context of economics and financial markets to describe a situation where a recession occurs in two distinct phases. This refers to cases where the economy appears to recover temporarily, only to decline again. Generally, there is a period of growth or recovery following the first downturn, but then the economy deteriorates again. This term is considered one of the significant patterns in stock market analysis or GDP growth rate evaluations, and it can serve as a warning signal for investors and policymakers.'

람다 함수를 사용하여 입력을 변환하는 방식은 스트리밍 작업과 호환되지 않을 수 있어 주의해야함

# 파이썬 파이프 연산자 오버로딩
파이썬에서 직접 | 연산자를 오버로딩하여 여러 작업을 순차적으로 처리하는 체인을 만들수 있음( __or__ )사용

In [10]:
class CustomLCEL:
    def __init__(self, value):
        self.value = value # 객체 생성 시 값을 초기화합니다.

    def __or__(self, other):
        if callable(other):
            # other가 함수일 경우, 함수를 호출하고 그 결과를 새로운 객체로 반환합니다.
            return CustomLCEL(other(self.value))
        else:
            # other가 함수가 아닐 경우, 오류를 발생시킵니다.
            raise ValueError("The right operand must be a callable function.")

    def result(self):
        # 현재 객체의 값을 반환합니다.
        return self.value

def add_exclamation(input):
    # 입력된 문자열에 느낌표를 추가합니다.
    return input + "!"

def reverse_string(input):
    # 입력된 문자열을 뒤집습니다.
    return input[::-1]

# 파이프라인을 생성하여 순차적으로 문자열 변환 작업을 수행합니다.
custom_chain = (
    CustomLCEL("랭체인 공부하기")
    | add_exclamation # 느낌표 추가
    | reverse_string # 문자열 뒤집기
)

# 최종 결과를 출력합니다.
result = custom_chain.result()
print(result)  # 출력: "!기하공부 체인랭"

!기하부공 인체랭


### 파이프 메서드
파이프 연산자 | 외에도 .pipe() 메서드로 각 작업을 순차적으로 연결할 수 있음

In [12]:
# (방법1) 여러 작업을 순차적으로 .pipe를 통해 연결하여 체인 구성하기
composed_chain_with_pipe = (
    # 이전에 정의된 체인(chain)으로 입력된 데이터를 받아옴
    chain
    # 입력된 데이터를 "answer" 키로 변환하는 람다 함수 적용
    .pipe(lambda input: {"answer": input})
    # analysis_prompt를 체인에 연결하여 설명을 영어로 번역하는 작업 추가
    .pipe(analysis_prompt)
    # 모델을 사용해 응답 생성
    .pipe(model)
    # 생성된 응답을 문자열로 파싱
    .pipe(StrOutputParser())
)
# "더블딥"이라는 주제로 체인을 실행하여 답변 생성
composed_chain_with_pipe.invoke({"topic": "더블딥"})

'"Double dip" is a term used in economics to describe a phenomenon where the economy temporarily recovers after a recession, only to experience another downturn. In other words, instead of a V-shaped recovery, it takes the form of a W, resulting in two separate recessions. This phenomenon can occur due to factors such as decreased consumer confidence, instability in the job market, or external economic influences. Double dips pose significant challenges for economic policymakers.'

In [13]:
# (방법 2) 좀 더 간단하게 연결하기
composed_chain_with_pipe = chain.pipe(lambda input: {"answer": input}, analysis_prompt, model, StrOutputParser())
# "더블딥"이라는 주제로 체인을 실행하여 답변 생성
composed_chain_with_pipe.invoke({"topic": "더블딥"})

'Double Dip is a term primarily used in economics to describe a situation where the economy recovers only to fall back into a recession. It refers to a scenario in which, after an initial downturn, the economy appears to recover for a while but then experiences another slowdown or contraction. This pattern typically indicates a temporary rebound in economic growth followed by a return to negative growth rates, which can negatively impact consumer confidence and investment. Such phenomena can be associated with deep economic recessions and become important considerations for policymakers and investors.'

### RunnableParallel을 이용한 체인 구성
RunnableParallel은 여러 개의 체인을 병렬로 실행하여 효율성을 높이는 데 유용함

In [ ]:
from langchain_core.runnables import RunnableParallel
#OPenAI 모델 초기화
model = ChatOpenAI()
# 한국어 설명 생성 프롬프트 체인
kor_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧은 설명을 해주세요.")
    | model
    | StrOutputParser()
)
# 영어 설명 생성 프롬프트 체인
eng_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧게 영어로 설명을 해주세요.")
    | model
    | StrOutputParser()
)
# 병렬 실행을 위한 RunnableParallel 설정
parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
# 주제에 대한 한국어와 영어 설명 생성
result = parallel_chain.invoke({"topic": "더블딥"})

# 결과 출력
print(result["kor"])
print(result["eng"])